# Web Scraping with lazy load

Page to be scraped: El Espectador

In [1]:
# Importing required libraries

import time

from bs4 import BeautifulSoup
from selenium import webdriver

from pymongo import MongoClient

In [2]:
# Creating a connection to MongoDB
client = MongoClient('', 27017)
db = client['news']
collection = db['elespectador']

ConfigurationError: Empty host (or extra comma in host list).

In [ ]:
# Base URL of the site to be analyzed
SITE_URL = 'https://www.elespectador.com'

In [ ]:
# Firefox web driver path
# Download the driver for you S.O. here: https://github.com/mozilla/geckodriver/releases
DRIVER_PATH = './geckodriver'

In [ ]:
# Creating a new firefox window
browser = webdriver.Firefox(executable_path = DRIVER_PATH)

In [ ]:
def make_request(browser, relative_path):
    # Making the request and rendering the browser
    browser.get(SITE_URL + relative_path)
    
    # Simulating vertical scrolling for handling lazy load
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis
    return BeautifulSoup(browser.page_source, 'html.parser')

In [ ]:
# Getting HTML content for a particular news listing page
soup = make_request(browser, '/archivo/politica/')

In [ ]:
# Finding the section where news are contained 
layout = soup.find(class_ = 'Layout-flexAds')

In [ ]:
# Getting blocks from layout
blocks = layout.find_all(class_ = 'Container Block', recursive = True)
print(len(blocks))

In [ ]:
# Finding and concatenating news cards
cards = blocks[0].find_all(class_ = 'Card_rowCardLeft') + blocks[1].find_all(class_ = 'Card_rowCardLeft')
len(cards)

In [ ]:
# Building a list with title and relative path of the news founded

news = []

for card in cards:
    news.append({
        'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
        'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
    })

In [ ]:
news

In [ ]:
for n in news:
    # Getting HTML content for each news page
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').find('a').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [ ]:
news

In [ ]:
# Storing extracted information for further analysis
collection.insert_many(news)